# Image classification using 4 animals Kaggle dataset

View dataset description and leaderboard [here](https://www.kaggle.com/competitions/4-animal-classification/)

## 1. Import modules

In [ ]:

import torch
from torchvision import transforms
from torch import nn
from tqdm import tqdm
from glob import glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Check GPU compatibility
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU device:", torch.cuda.get_device_name(0))
    print("GPU memory:", round(torch.cuda.get_device_properties(0).total_memory/1024**3),"GB")
else:
    print("GPU is not available. Using CPU")
    
CUDA = torch.cuda.is_available()
device = "cuda" if CUDA else "cpu"


## 2. Process data

In [ ]:

# load data
test_data =[]
train_data_x = []
train_data_y = []

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((244,244)),
])

for file in glob("./data/4-animal-classification/test/test/*"):
    img = Image.open(file)
    img = transform(img)
    test_data.append(img)
    

animals = ['cat','deer','dog','horse']

for label, animal in enumerate(animals):
    for file in glob(f"./data/4-animal-classification/train/{animal}/*"):
        img = Image.open(file)
        img = transform(img)
        train_data_x.append(img)
        train_data_y.append(label)


In [ ]:
# convert to tensor
test_data = torch.tensor(np.array(test_data),dtype=torch.float32)
train_data_x = torch.tensor(np.array(train_data_x),dtype=torch.float32)
train_data_y = torch.tensor(np.array(train_data_y),dtype=torch.long)

# one_hot encode labels
train_data_y = nn.functional.one_hot(train_data_y,4)
train_data_y = train_data_y.to(torch.float32)

In [ ]:
print("Train data shape:",train_data_x.shape)
print("Train label shape:",train_data_y.shape)
print("Test data shape:",test_data.shape)

In [ ]:
# save data 
torch.save(train_data_x, "./data/4-animal-classification/train_data_x.pt")
torch.save(train_data_y, "./data/4-animal-classification/train_data_y.pt")
torch.save(test_data, "./data/4-animal-classification/test_data.pt")


# 3. Define architectures

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.sequential_224 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.sequential_112 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.sequential_56 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same'), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.sequential_28 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.sequential_14 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.sequential_linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512*7*7, out_features=2**12), nn.ReLU(),
            nn.Linear(in_features=2**12, out_features=2**11), nn.ReLU(),
            nn.Linear(in_features=2**11, out_features=num_classes), nn.ReLU(),
            nn.Softmax(dim=1)
        )

    def forward(self,x):
        x = self.sequential_224(x)
        x = self.sequential_112(x)
        x = self.sequential_56(x)
        x = self.sequential_28(x)
        x = self.sequential_14(x)
        x = self.sequential_linear(x)
        return x

## 4. Train models

In [ ]:

# load train data
train_data_x = torch.load("./data/4-animal-classification/train_data_x.pt",mmap=True)
train_data_y = torch.load("./data/4-animal-classification/train_data_y.pt",mmap=True)

In [ ]:
# define data loader
batch_size = 6

train_idx, val_idx = train_test_split(np.arange(train_data_x.shape[0]), test_size=0.25,shuffle=True,random_state=42)

class Data(Dataset):
    def __init__(self,data,label,idx_list):
        self.data = data
        self.label = label
        self.idx_list = idx_list
    def __len__(self):
        return self.idx_list.shape[0]
    def __getitem__(self,idx):
        return self.data[self.idx_list[idx]], self.label[self.idx_list[idx]]


train_data = Data(train_data_x,train_data_y,train_idx)
val_data = Data(train_data_x,train_data_y,val_idx)

train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_data,batch_size=batch_size,shuffle=True)


In [ ]:
# Initialize model and otimizer
model = VGG16(num_classes=4)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = model.to(device)
loss_fn = loss_fn.to(device)

for epoch in range(50):
    train_loss = 0 
    val_loss = 0
    model.train()
    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = y.to(device)
        
        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(val_loader):
            x = x.to(device)
            y = y.to(device)
            
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            val_loss += loss.item()
    print(f"Epoch {epoch} | Train Loss: {train_loss} | Val Loss: {val_loss}")
    